In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle

from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP

np.random.seed(1337)
random.seed(1337)

# make up a dataset

from sklearn.datasets import make_moons, make_blobs
X, y = make_moons(n_samples=100, noise=0.1)

y = y*2 - 1 # make y be -1 or 1

# initialize a model 
model = MLP(2, [16, 16, 1]) # 2-layer neural network


# loss function
def loss(batch_size=None):
    
    # inline DataLoader :)
    if batch_size is None:
        Xb, yb = X, y
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X[ri], y[ri]
        
    inputs = [list(map(Value, xrow)) for xrow in Xb]
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss
    
    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

# IBP Analysis on untrained network for a single input
#custom_x0 = 1
#custom_x1 = 0
#eps = 0.1
#input_with_bounds = [Value(custom_x0), Value(custom_x1)]
#input_with_bounds[0].lower = input_with_bounds[0].data - eps
#input_with_bounds[0].upper = input_with_bounds[0].data + eps
#input_with_bounds[1].lower = input_with_bounds[1].data - eps
#input_with_bounds[1].upper = input_with_bounds[1].data + eps
#score = model(input_with_bounds)
#score.ibp()
#print(f"Output bounds for custom input (untrained): lower={score.lower}, upper={score.upper}")

# optimization
for k in range(100):
    # forward
    total_loss, acc = loss()
    # backward
    model.zero_grad()
    total_loss.backward()
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    #if k % 1 == 0:
    #print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")
    
print(f"loss {total_loss.data}, accuracy {acc*100}%")
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [1]:
# Load model from pickle file
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import copy
import sys
from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP
import cvxpy as cp

np.random.seed(1337)
random.seed(1337)


with open('model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
    
def computebounds(custom_x0, custom_x1, eps, model):
    input_with_bounds = [Value(custom_x0), Value(custom_x1)]
    input_with_bounds[0].lower = input_with_bounds[0].data - eps
    input_with_bounds[0].upper = input_with_bounds[0].data + eps
    input_with_bounds[1].lower = input_with_bounds[1].data - eps
    input_with_bounds[1].upper = input_with_bounds[1].data + eps

    score = model(input_with_bounds)
    score.ibp()
    return score

# Collect all ReLU nodes in the computation graph of the last score
def collect_relu_nodes(output_node):
    relu_nodes = []
    visited = set()
    def traverse(v):
        if v not in visited:
            visited.add(v)
            if v._op == 'ReLU':
                # Only add if lower and upper bounds have a sign change
                relu_input = list(v._prev)[0]
                lower = relu_input.lower
                upper = relu_input.upper
                if lower is not None and upper is not None and lower * upper < 0:
                    relu_nodes.append(v)
            for child in getattr(v, '_prev', []):
                traverse(child)
    traverse(output_node)
    return relu_nodes

  
#score = computebounds(2, 0, 0.1, loaded_model)     
#relu_nodes = collect_relu_nodes(score) - debugging


def branch_and_bound(score):
    
    # Collect ReLU nodes
    relu_nodes = collect_relu_nodes(score)
    
    # No ReLU nodes with sign change in bounds found
    if not relu_nodes:
        return score.lower, score.upper
    
    # Pick a ReLU node at random to branch on
    chosen_relu = random.choice(relu_nodes)
    relu_input = list(chosen_relu._prev)[0]
    
    
    # Branch 1: ReLU input >= 0
    score_branch1 = copy.deepcopy(score)
    relu_input_pos = find_corresponding_node(score_branch1, relu_input)
    relu_input_pos.lower = 0
    relu_input_pos.upper = relu_input_pos.upper
    score_branch1.ibp()
    bounds1 = branch_and_bound(score_branch1)

    # Branch 2: ReLU input <= 0
    score_branch2 = copy.deepcopy(score)
    relu_input_neg = find_corresponding_node(score_branch2, relu_input)
    relu_input_neg.lower = relu_input_neg.lower
    relu_input_neg.upper = 0
    score_branch2.ibp()
    bounds2 = branch_and_bound(score_branch2)

    # Return global bounds
    return min(bounds1[0], bounds2[0]), max(bounds1[1], bounds2[1])
   
def find_corresponding_node(new_score, old_node):
    visited = set()
    stack = [new_score]
    while stack:
        v = stack.pop()
        if v.id == old_node.id:
            return v
        visited.add(v)
        for child in v._prev:
            if child not in visited:
                stack.append(child)
    raise ValueError("Corresponding node not found")


def planet_relaxation(score):
    # Collect ReLU nodes
    relu_nodes = collect_relu_nodes(score)
    
    if not relu_nodes:
        return score.lower, score.upper
    
    # Create a list of constraints
    constraints = []
    
    for relu_node in relu_nodes:
        relu_input = list(relu_node._prev)[0]
        lower_bound = relu_input.lower
        upper_bound = relu_input.upper
        
        if lower_bound is not None and upper_bound is not None:
            # Add constraints for the ReLU node
            constraints.append(cp.constraints.NonPos(relu_input - 0))
    
    # Define the optimization problem
    objective = cp.Minimize(0)  # No specific objective, just bounds
    prob = cp.Problem(objective, constraints)
    
    # Solve the problem
    prob.solve()
    
    return score.lower, score.upper


In [2]:
score = computebounds(0, 0.5, 0.1, loaded_model)
print(score.lower, score.upper)
relu_nodes_test = collect_relu_nodes(score)
print(f"Number of ReLU nodes with sign change in bounds: {len(relu_nodes_test)}")

-2.1302737934450184 1.3347824065425349
Number of ReLU nodes with sign change in bounds: 9


In [3]:

score = computebounds(0, 0.5, 0.1, loaded_model)
global_lower, global_upper = branch_and_bound(score)
print(f"Global lower bound: {global_lower}, Global upper bound: {global_upper}")

Global lower bound: -2.1302737934450184, Global upper bound: 1.3347824065425349
